# Solving Linear Equations using A=LU
ビデオでは触れられていませんでしたが、教科書の第2章の最後の方に Matrix Factorization を使って線形方程式を解くMATLABコードが載っていたので、実際にPythonで試してみました。

方針としては、Augmented Matrixを使った例のようにAx=bを一度に解くのではなく、[1]行列AのLU分解, [2]Lによるｂのforward elimination, [3]Uによるｘのback substitution,の3段階に分けて解きます。一見手間が増えているように感じますが、この方法だとAとbを分けて考えることができ、一度AのLU分解を行ってしまえばbが変わっても計算を最初からやり直す必要がないという点で優れているのだそうです。

$A=LU, Lc=b, Ux=c$が$Ax=b$と等価であることは、$Ux=c$の両辺にLをかけると$LUx=Lc$になることからわ

ビデオだとMatrix Factorizationが線形方程式を解くこととどのように関連しているのかについての記述が薄いように感じますが、実際にはA=LUが第2章のタイトルである"Solving Linear Equations"をコンピュータ上で実装する上で欠かせない考え方であることが分かります。

In [1]:
# Python だとMATLABのようにデフォルトで行列を扱えるわけではないので、numpyモジュールをインポートしました。
import numpy as np

今回のソルバーはAx=bのAに対して以下の条件を要求します。2番めの条件に関してはPermutation Matrixを取り入れればクリアできるはずなので、また時間のあるときに改修します。
+ A は正方行列
+ A の対角要素は0でない

In [2]:
# Factorization A=LU
def slu(A_):
    # 元コードだと行列Aに関して副作用があるので,引数A_を書き換えないようにコピーを作りました
    A = A_.copy()
    # Aが正方行列かどうかを確認（列数と行数が一致するか）
    if (A.shape[0] != A.shape[1]):
        print("A is not square matrix!")
        return False
    n = len(A) # number of dimension
    tol = 1.0e-6 # pivotが0かどうかを判定する基準
    L = np.zeros(A.size).reshape(n,n) # A.sizeはAの要素数を返します.
    U = np.zeros(A.size).reshape(n,n) # reshapeでvectorをmatrix[n,n]に変換.
    cost = 0 #計算量カウンター
    for k in range(n):
        # pivot が0でないかの確認
        if np.abs(A[k,k]) < tol:
            print("A is not invertible!")
            return False
        L[k,k] = 1 # 下三角行列の対角要素はすべて1
        for i in range(k+1,n):
            L[i,k]=A[i,k]/A[k,k] # Elmination matrix の倍率l_{i,k}を求める
            for j in range(k+1,n):
                A[i,j] = A[i,j] - L[i,k]*A[k,j] # ここでAの対角要素も変化するのが重要
                cost += 1
        for j in range(k,n): # 上三角行列は対角線より下はすべて0なので計算しない
            U[k,j] = A[k,j]
            cost +=1
    print("factorization cost:{0}".format(cost))
    return [L, U]

In [5]:
# 教科書p98 Example 3の例で試しました
A = np.matrix([[1,2],[4,9]])
b = np.array([5,21])

In [6]:
[L,U] = slu(A)
print("L={0}, \nU={1}".format(L,U))

factorization cost:4
L=[[ 1.  0.]
 [ 4.  1.]], 
U=[[ 1.  2.]
 [ 0.  1.]]


↑　行列のLU分解のテスト。手計算の結果と合ってます

In [7]:
def slv(A_, b_):
    # まず最初にAをLUに分解する
    [L,U] = slu(A_)
    b = b_.copy() # 副作用対策
    s = 0 # Forward elimination でターゲットとなる対角要素より左側にある値の合計値
    t = 0 # Back substitution でターゲットとなる対角要素より右側にある値の合計値
    n = len(A_) # number of dimension of A
    c = np.zeros(n) # Lc = bの解c
    x = np.zeros(n) # solution x
    cost = 0 #計算量カウンター
    # Forward elimination to solve Lc = b
    for k in range(n): # Lは下三角行列なのでForward
        for j in range(k):
            s = s + L[k,j]*c[j]
            cost += 1
        c[k] = b[k] - s # Lの対角要素は1なので割り算は不要
        s = 0
    # Back substitution to solve Ux = c
    for k in reversed(range(n)): # Uは上三角行列なのでBackward
        for j in range(k+1, n):
            t = t + U[k,j]*x[j]
            cost += 1
        x[k] = (c[k]-t)/U[k,k] # Uの対角要素は1とは限らないので割り算必要
        t = 0
    print("cost of right sides:{0}".format(cost))
    return x.T

In [8]:
slv(A,b)

factorization cost:4
cost of right sides:2


array([ 3.,  1.])

In [9]:
# 3 by 3 の行列でも試してみました
A = np.matrix([[1,1,1],[1,2,3],[1,3,6]])
b = np.array([5,7,11])

In [10]:
slv(A,b)

factorization cost:11
cost of right sides:6


array([ 5., -2.,  2.])